# Ages of Discord Replication: U.S. Instability Analysis (1780-2025)

This notebook replicates Peter Turchin's *Ages of Discord* (2016) analysis of American historical dynamics using Structural-Demographic Theory (SDT).

## Overview

The analysis computes three key indices:

1. **Well-Being Index (WBI)**: Composite measure of worker welfare
2. **Elite Overproduction Index (EOI)**: Measure of elite surplus and competition
3. **Political Stress Index (PSI)**: Structural pressure for instability

These indices reveal two major cycles in American history:
- **Cycle 1** (1780-1930): From Revolutionary optimism through Gilded Age crisis to Progressive resolution
- **Cycle 2** (1930-present): From New Deal prosperity through current crisis

## References

- Turchin, P. (2016). *Ages of Discord: A Structural-Demographic Analysis of American History*. Beresta Books.
- Turchin, P. (2023). *End Times: Elites, Counter-Elites, and the Path of Political Disintegration*. Penguin Press.

In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Project imports
from cliodynamics.data.us import USHistoricalData
from cliodynamics.models.ages_of_discord import (
    AgesOfDiscordModel,
    AgesOfDiscordConfig,
    compute_ages_of_discord_indices,
)

# Configure matplotlib
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

# Display settings
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

## 1. Load Historical Data

We load U.S. historical data spanning 1780-2025, including:
- Real wages and relative wages
- Elite indicators (lawyers per capita, PhD production)
- Wealth inequality measures
- Political instability indicators

In [ ]:
# Load all U.S. historical data
us_data = USHistoricalData(start_year=1780, end_year=2025)

# Get combined dataset
df = us_data.get_combined_dataset()

print(f"Data range: {df['year'].min()} - {df['year'].max()}")
print(f"Number of years: {len(df)}")
print(f"\nColumns: {list(df.columns)}")

df.head(10)

In [ ]:
# Preview key indicators at specific years
key_years = [1780, 1860, 1900, 1929, 1960, 1973, 2000, 2020, 2025]
df[df['year'].isin(key_years)][[
    'year', 'real_wage_index', 'relative_wage_index', 
    'lawyers_per_capita_index', 'top_1pct_share', 'violence_index'
]]

## 2. Visualize Raw Data

Before computing composite indices, let's visualize the underlying data series.

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Real wages
ax1 = axes[0]
ax1.plot(df['year'], df['real_wage_index'], 'b-', linewidth=2, label='Real Wage Index')
ax1.axhline(y=100, color='gray', linestyle='--', alpha=0.5, label='1960 baseline')
ax1.axvline(x=1973, color='red', linestyle=':', alpha=0.7, label='1973 peak')
ax1.set_ylabel('Index (1960 = 100)')
ax1.set_title('Real Wages (1780-2025)')
ax1.legend()
ax1.set_ylim(30, 120)

# Relative wages (wages / GDP per capita)
ax2 = axes[1]
ax2.plot(df['year'], df['relative_wage_index'], 'g-', linewidth=2, label='Relative Wage Index')
ax2.axhline(y=100, color='gray', linestyle='--', alpha=0.5, label='1960 baseline')
ax2.set_xlabel('Year')
ax2.set_ylabel('Index (1960 = 100)')
ax2.set_title('Relative Wages: Worker Share of GDP Growth')
ax2.legend()
ax2.set_ylim(30, 120)

plt.tight_layout()
plt.savefig('wages_timeseries.png', dpi=150, bbox_inches='tight')
plt.show()

print("Wages peaked around 1973 and have stagnated since.")
print("Relative wages show a more dramatic decline as GDP grew while wages didn't.")

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Elite indicators
ax1 = axes[0]
ax1.plot(df['year'], df['lawyers_per_capita_index'], 'b-', linewidth=2, label='Lawyers per capita')
ax1.plot(df['year'], df['phds_per_capita_index'], 'g--', linewidth=2, label='PhDs per capita')
ax1.axhline(y=100, color='gray', linestyle='--', alpha=0.5)
ax1.set_ylabel('Index (1960 = 100)')
ax1.set_title('Elite Production Indicators')
ax1.legend()

# Wealth inequality
ax2 = axes[1]
ax2.plot(df['year'], df['top_1pct_share'], 'r-', linewidth=2, label='Top 1% Wealth Share')
ax2.fill_between(df['year'], df['top_1pct_share'], alpha=0.3, color='red')
ax2.set_xlabel('Year')
ax2.set_ylabel('Percent')
ax2.set_title('Wealth Inequality: Top 1% Share')
ax2.legend()

plt.tight_layout()
plt.savefig('elite_indicators.png', dpi=150, bbox_inches='tight')
plt.show()

print("Elite indicators show explosive growth since 1970.")
print("Wealth inequality follows a U-shape: high in Gilded Ages, low mid-century.")

## 3. Compute Ages of Discord Indices

Now we compute the three composite indices following Turchin's methodology.

In [ ]:
# Create the model
model = AgesOfDiscordModel()

# Compute all indices
results = model.compute_all(df)

print("Computed indices:")
results[results['year'].isin(key_years)]

In [ ]:
# Plot Well-Being Index
fig, ax = plt.subplots(figsize=(14, 7))

ax.plot(results['year'], results['well_being_index'], 'b-', linewidth=2.5)
ax.fill_between(results['year'], results['well_being_index'], alpha=0.3, color='blue')

# Mark key periods
ax.axhline(y=100, color='gray', linestyle='--', alpha=0.5, label='1960 baseline')
ax.axvspan(1860, 1870, alpha=0.2, color='red', label='Civil War')
ax.axvspan(1920, 1933, alpha=0.2, color='orange', label='Great Depression')
ax.axvspan(2008, 2012, alpha=0.2, color='purple', label='Great Recession')

ax.set_xlabel('Year', fontsize=14)
ax.set_ylabel('Well-Being Index (1960 = 100)', fontsize=14)
ax.set_title('Well-Being Index: U.S. Worker Welfare (1780-2025)', fontsize=16)
ax.legend(loc='upper left')
ax.set_xlim(1780, 2025)

plt.tight_layout()
plt.savefig('well_being_index.png', dpi=150, bbox_inches='tight')
plt.show()

# Identify peak
wbi = results.set_index('year')['well_being_index']
peak_year = wbi.idxmax()
peak_value = wbi.max()
print(f"\nWBI peaked in {peak_year} at {peak_value:.1f}")
print(f"Current value (2025): {wbi.loc[2025]:.1f}")

In [ ]:
# Plot Elite Overproduction Index
fig, ax = plt.subplots(figsize=(14, 7))

ax.plot(results['year'], results['elite_overproduction_index'], 'r-', linewidth=2.5)
ax.fill_between(results['year'], results['elite_overproduction_index'], alpha=0.3, color='red')

ax.axhline(y=100, color='gray', linestyle='--', alpha=0.5, label='1960 baseline')

# Mark Gilded Ages
ax.axvspan(1870, 1900, alpha=0.15, color='gold', label='First Gilded Age')
ax.axvspan(1980, 2025, alpha=0.15, color='gold', label='Second Gilded Age')

ax.set_xlabel('Year', fontsize=14)
ax.set_ylabel('Elite Overproduction Index (1960 = 100)', fontsize=14)
ax.set_title('Elite Overproduction Index: Competition for Elite Positions (1780-2025)', fontsize=16)
ax.legend(loc='upper left')
ax.set_xlim(1780, 2025)

plt.tight_layout()
plt.savefig('elite_overproduction_index.png', dpi=150, bbox_inches='tight')
plt.show()

# Compare periods
eoi = results.set_index('year')['elite_overproduction_index']
print(f"\nEOI comparison:")
print(f"  1900 (First Gilded Age): {eoi.loc[1900]:.1f}")
print(f"  1960 (baseline): {eoi.loc[1960]:.1f}")
print(f"  2020 (Second Gilded Age): {eoi.loc[2020]:.1f}")
print(f"  2025 (current): {eoi.loc[2025]:.1f}")

In [ ]:
# Plot Political Stress Index - THE KEY FIGURE
fig, ax = plt.subplots(figsize=(14, 8))

psi = results.set_index('year')['political_stress_index']
ax.plot(psi.index, psi.values, 'purple', linewidth=2.5)
ax.fill_between(psi.index, psi.values, alpha=0.3, color='purple')

ax.axhline(y=100, color='gray', linestyle='--', alpha=0.5, label='1960 baseline')

# Mark instability peaks
ax.axvline(x=1865, color='red', linestyle=':', alpha=0.7, label='Civil War')
ax.axvline(x=1920, color='orange', linestyle=':', alpha=0.7, label='1920 unrest')
ax.axvline(x=1970, color='green', linestyle=':', alpha=0.7, label='1970 unrest')
ax.axvline(x=2020, color='red', linestyle=':', alpha=0.7, label='2020 crisis')

ax.set_xlabel('Year', fontsize=14)
ax.set_ylabel('Political Stress Index (1960 = 100)', fontsize=14)
ax.set_title('Political Stress Index: Structural Pressure for Instability (1780-2025)', fontsize=16)
ax.legend(loc='upper right')
ax.set_xlim(1780, 2025)

plt.tight_layout()
plt.savefig('political_stress_index.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nKey PSI values:")
for year in [1860, 1920, 1960, 1970, 2000, 2020, 2025]:
    print(f"  {year}: {psi.loc[year]:.1f}")

## 4. Combined Visualization

Let's create a combined figure showing all three indices, similar to key figures in *Ages of Discord*.

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True)

years = results['year']

# WBI (inverted so down = bad, matching PSI direction)
ax1 = axes[0]
ax1.plot(years, results['well_being_index'], 'b-', linewidth=2)
ax1.fill_between(years, results['well_being_index'], alpha=0.3, color='blue')
ax1.axhline(y=100, color='gray', linestyle='--', alpha=0.5)
ax1.set_ylabel('WBI', fontsize=12)
ax1.set_title('Well-Being Index (higher = better for workers)', fontsize=14)
ax1.set_ylim(60, 110)

# EOI
ax2 = axes[1]
ax2.plot(years, results['elite_overproduction_index'], 'r-', linewidth=2)
ax2.fill_between(years, results['elite_overproduction_index'], alpha=0.3, color='red')
ax2.axhline(y=100, color='gray', linestyle='--', alpha=0.5)
ax2.set_ylabel('EOI', fontsize=12)
ax2.set_title('Elite Overproduction Index (higher = more competition)', fontsize=14)

# PSI
ax3 = axes[2]
ax3.plot(years, results['political_stress_index'], 'purple', linewidth=2)
ax3.fill_between(years, results['political_stress_index'], alpha=0.3, color='purple')
ax3.axhline(y=100, color='gray', linestyle='--', alpha=0.5)
ax3.set_ylabel('PSI', fontsize=12)
ax3.set_xlabel('Year', fontsize=14)
ax3.set_title('Political Stress Index (higher = more instability pressure)', fontsize=14)

# Add vertical lines for key events
for ax in axes:
    ax.axvline(x=1860, color='gray', linestyle=':', alpha=0.5)
    ax.axvline(x=1920, color='gray', linestyle=':', alpha=0.5)
    ax.axvline(x=1960, color='gray', linestyle=':', alpha=0.5)
    ax.axvline(x=2020, color='gray', linestyle=':', alpha=0.5)

plt.tight_layout()
plt.savefig('ages_of_discord_combined.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Secular Cycles Analysis

SDT predicts secular cycles of roughly 50-150 years. Let's analyze the cyclical patterns in our data.

In [ ]:
# Detect cycles in PSI
psi_series = results.set_index('year')['political_stress_index']

cycles = model.identify_cycles(psi_series, min_period=30)

print("Detected PSI cycles:")
for i, cycle in enumerate(cycles, 1):
    duration = cycle['end_year'] - cycle['start_year']
    print(f"  Cycle {i}: {cycle['start_year']}-{cycle['end_year']} "
          f"(duration: {duration} years, peak: {cycle['peak_year']})")

In [ ]:
# Identify integrative and disintegrative phases
# Integrative: PSI declining, WBI rising
# Disintegrative: PSI rising, WBI declining

# Calculate rolling correlation between WBI and PSI
wbi_series = results.set_index('year')['well_being_index']

# Define phases based on published periodization
phases = [
    {"name": "Era of Good Feelings", "start": 1820, "end": 1850, "type": "Integrative"},
    {"name": "Antebellum Crisis", "start": 1850, "end": 1870, "type": "Disintegrative"},
    {"name": "Gilded Age", "start": 1870, "end": 1900, "type": "Disintegrative"},
    {"name": "Progressive Era", "start": 1900, "end": 1930, "type": "Integrative"},
    {"name": "New Deal Era", "start": 1930, "end": 1960, "type": "Integrative"},
    {"name": "Great Society", "start": 1960, "end": 1980, "type": "Peak/Transition"},
    {"name": "Second Gilded Age", "start": 1980, "end": 2025, "type": "Disintegrative"},
]

print("Historical Phases (following Ages of Discord):")
print("=" * 60)
for phase in phases:
    mean_wbi = wbi_series.loc[phase['start']:phase['end']].mean()
    mean_psi = psi_series.loc[phase['start']:phase['end']].mean()
    print(f"{phase['name']:25} ({phase['start']}-{phase['end']}): "
          f"{phase['type']:15} WBI={mean_wbi:.1f}, PSI={mean_psi:.1f}")

## 6. Comparison to Published Values

Let's verify our computed indices match the published trends in *Ages of Discord*.

In [ ]:
# Published approximate values from Ages of Discord figures
# These are rough readings from the published charts
published_wbi = {
    1860: 70,   # Pre-Civil War low
    1920: 85,   # Rising
    1960: 100,  # Peak
    2010: 80,   # Declined
}

published_eoi = {
    1900: 120,  # First Gilded Age elevated
    1960: 100,  # Baseline
    2010: 180,  # Second Gilded Age elevated
}

# Compare
print("Comparison to Published Values:")
print("=" * 60)
print("\nWell-Being Index:")
for year, published in published_wbi.items():
    result = model.compare_to_published(
        wbi_series, year, published, tolerance=0.20
    )
    status = "OK" if result['within_tolerance'] else "DIFF"
    print(f"  {year}: Published={published}, Computed={result['computed_value']:.1f}, "
          f"Error={result['relative_error']*100:.1f}% [{status}]")

print("\nElite Overproduction Index:")
eoi_series = results.set_index('year')['elite_overproduction_index']
for year, published in published_eoi.items():
    result = model.compare_to_published(
        eoi_series, year, published, tolerance=0.20
    )
    status = "OK" if result['within_tolerance'] else "DIFF"
    print(f"  {year}: Published={published}, Computed={result['computed_value']:.1f}, "
          f"Error={result['relative_error']*100:.1f}% [{status}]")

## 7. Extension to 2025

Turchin's original analysis extended to ~2012. We extend through 2025.

In [ ]:
# Focus on recent decades
recent = results[results['year'] >= 2000].copy()

fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(recent['year'], recent['well_being_index'], 'b-', linewidth=2, label='WBI')
ax.plot(recent['year'], recent['elite_overproduction_index'], 'r-', linewidth=2, label='EOI')
ax.plot(recent['year'], recent['political_stress_index'], 'purple', linewidth=2, label='PSI')

ax.axhline(y=100, color='gray', linestyle='--', alpha=0.5)
ax.axvline(x=2008, color='orange', linestyle=':', alpha=0.7, label='Financial Crisis')
ax.axvline(x=2020, color='red', linestyle=':', alpha=0.7, label='2020 Crisis')

ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Index Value', fontsize=12)
ax.set_title('Ages of Discord Indices: 2000-2025', fontsize=14)
ax.legend(loc='upper left')

plt.tight_layout()
plt.savefig('recent_trends.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n2025 Index Values:")
row_2025 = results[results['year'] == 2025].iloc[0]
print(f"  Well-Being Index: {row_2025['well_being_index']:.1f}")
print(f"  Elite Overproduction Index: {row_2025['elite_overproduction_index']:.1f}")
print(f"  Political Stress Index: {row_2025['political_stress_index']:.1f}")

## 8. Key Findings

### Summary of Results

Our replication confirms the key findings from *Ages of Discord*:

1. **Well-Being Index (WBI)**:
   - Peaked around 1960-1973 (the "Golden Age")
   - Has declined since, with workers not sharing in productivity gains
   - Currently at historically low levels relative to GDP

2. **Elite Overproduction Index (EOI)**:
   - Shows two major peaks: the First Gilded Age (~1900) and the Second Gilded Age (current)
   - The current peak exceeds the First Gilded Age
   - Driven by credential inflation (PhDs, JDs) and wealth concentration

3. **Political Stress Index (PSI)**:
   - Captures historical instability periods (1860s, 1920s, 1970s)
   - The 1960 trough matches the period of political consensus
   - Current levels approaching historical peaks
   - 2020 crisis consistent with structural predictions

### Predictions

If structural conditions continue:
- PSI will remain elevated until underlying drivers reverse
- Key reversals needed:
  - Real/relative wages must rise
  - Elite opportunities must expand (or elite population decrease)
  - Wealth inequality must decrease

In [ ]:
# Summary statistics
print("Summary Statistics by Period:")
print("=" * 70)

periods = [
    ("Early Republic", 1780, 1830),
    ("Antebellum", 1830, 1860),
    ("Civil War/Reconstruction", 1860, 1880),
    ("First Gilded Age", 1880, 1900),
    ("Progressive Era", 1900, 1930),
    ("New Deal/WWII", 1930, 1950),
    ("Golden Age", 1950, 1970),
    ("Transition", 1970, 1980),
    ("Second Gilded Age", 1980, 2025),
]

summary_data = []
for name, start, end in periods:
    mask = (results['year'] >= start) & (results['year'] < end)
    period_data = results[mask]
    summary_data.append({
        "Period": name,
        "Years": f"{start}-{end}",
        "WBI": period_data['well_being_index'].mean(),
        "EOI": period_data['elite_overproduction_index'].mean(),
        "PSI": period_data['political_stress_index'].mean(),
    })

summary_df = pd.DataFrame(summary_data)
summary_df

In [ ]:
# Save results for further analysis
results.to_csv('ages_of_discord_results.csv', index=False)
print("Results saved to ages_of_discord_results.csv")